In [1]:
from db_tools.setup import setup
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

In [2]:
spark = setup()

23/10/28 19:50:53 WARN Utils: Your hostname, luan-Dell-G15-5520 resolves to a loopback address: 127.0.1.1; using 192.168.1.12 instead (on interface wlp0s20f3)
23/10/28 19:50:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/10/28 19:50:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/28 19:50:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


SparkSession available as "spark"


In [3]:
def show_query(query,SparkSession = spark,n = 20):
    return SparkSession.sql(query).show()

# CTE Examples

## Classic example

In [7]:
query = """
WITH cte_film AS (
    SELECT 
        film_id, 
        title,
        (CASE 
            WHEN length < 30 THEN 'Short'
            WHEN length < 90 THEN 'Medium'
            ELSE 'Long'
        END) length    
    FROM
        film
)
SELECT
    film_id,
    title,
    length
FROM 
    cte_film
WHERE
    length = 'Long'
ORDER BY 
    title; 
"""
show_query(query)

+-------+--------------------+------+
|film_id|               title|length|
+-------+--------------------+------+
|      4|    Affair Prejudice|  Long|
|      5|         African Egg|  Long|
|      6|        Agent Truman|  Long|
|      9|       Alabama Devil|  Long|
|     11|     Alamo Videotape|  Long|
|     12|      Alaska Phantom|  Long|
|     13|         Ali Forever|  Long|
|     14|      Alice Fantasia|  Long|
|     16|     Alley Evolution|  Long|
|     19|        Amadeus Holy|  Long|
|     21|     American Circus|  Long|
|     23|Anaconda Confessions|  Long|
|     24|    Analyze Hoosiers|  Long|
|     27|     Anonymous Human|  Long|
|     28|         Anthem Luke|  Long|
|     29|  Antitrust Tomatoes|  Long|
|     31|       Apache Divine|  Long|
|     32|Apocalypse Flamingos|  Long|
|     33|         Apollo Teen|  Long|
|     35|Arachnophobia Rol...|  Long|
+-------+--------------------+------+
only showing top 20 rows



## CTE and JOIN

In [8]:
query = """
WITH cte_rental AS (
    SELECT staff_id,
        COUNT(rental_id) rental_count
    FROM   rental
    GROUP  BY staff_id
)
SELECT s.staff_id,
    first_name,
    last_name,
    rental_count
FROM staff s
    INNER JOIN cte_rental USING (staff_id); 
"""
show_query(query)

+--------+----------+---------+------------+
|staff_id|first_name|last_name|rental_count|
+--------+----------+---------+------------+
|       1|      Mike|  Hillyer|        8040|
|       2|       Jon| Stephens|        8004|
+--------+----------+---------+------------+



## CTE and windowing

In [9]:
query = """
WITH cte_film AS  (
    SELECT film_id,
        title,
        rating,
        length,
        RANK() OVER (
            PARTITION BY rating
            ORDER BY length DESC) 
        length_rank
    FROM 
        film
)
SELECT *
FROM cte_film
WHERE length_rank = 1;
"""
show_query(query)

+-------+------------------+------+------+-----------+
|film_id|             title|rating|length|length_rank|
+-------+------------------+------+------+-----------+
|    182|    Control Anthem|     G|   185|          1|
|    212|    Darn Forrester|     G|   185|          1|
|    609|     Muscle Bright|     G|   185|          1|
|    198|  Crystal Breaking| NC-17|   184|          1|
|    499|    King Evolution| NC-17|   184|          1|
|    820|    Sons Interview| NC-17|   184|          1|
|    821|    Sorority Queen| NC-17|   184|          1|
|    991|      Worst Banger|    PG|   185|          1|
|    141|     Chicago North| PG-13|   185|          1|
|    349|       Gangs Pride| PG-13|   185|          1|
|    690|      Pond Seattle| PG-13|   185|          1|
|    426|         Home Pity|     R|   185|          1|
|    817|Soldiers Evolution|     R|   185|          1|
|    872| Sweet Brotherhood|     R|   185|          1|
+-------+------------------+------+------+-----------+

